In [1]:
from pulp import *   # PuLP - Python Library for Linear Programming
import pandas as pd
import numpy as np

In [2]:
n_supply = 100   #number of supply points
n_demand = 4096  #number of demand points

#Importing csv files 
demand_set = pd.read_csv('Demand_Predictions.csv') 
supply_set = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
dist_mat=pd.read_csv('Distance_Matrix.csv')

In [3]:
dist_mat=dist_mat.to_numpy()
dist_mat

array([[53.14215611, 52.20883581, 51.27802958, ..., 45.52225335,
        45.78153919, 46.06108022],
       [68.38785811, 67.8544489 , 67.33166666, ..., 24.75540075,
        25.73241079, 26.71112248],
       [73.39766971, 72.77224371, 72.15525616, ..., 15.95179219,
        16.88316352, 17.8219737 ],
       ...,
       [41.81242567, 40.96146071, 40.11737262, ..., 48.39291434,
        48.92276036, 49.46714873],
       [56.71278449, 55.96843837, 55.23216692, ..., 31.44214632,
        32.04270174, 32.66283673],
       [19.37856861, 18.39424899, 17.41171648, ..., 72.91708351,
        73.49639816, 74.08468139]])

In [4]:
demand_set

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,15.088540,16.925371
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,13.453829,13.827596
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,17.501605,21.635659
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,18.674598,22.179762
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,20.526166,25.623670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,6.540436,7.850964
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.459605,3.065697
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,3.785937,4.543876
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,7.283772,7.752700


In [5]:
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,3
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4
96,96,30.318396,33.388335,32,5,10
97,97,36.218839,22.235766,32,4,14
98,98,42.936915,38.122442,28,7,5


In [6]:
col=[]
for i in range(supply_set.shape[0]):
    ans=supply_set.iloc[i:i+1,4].values*200+supply_set.iloc[i:i+1,5].values*400
    col.append(int(ans))
col=np.array(col)
supply_set['Smax18']=col
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Smax18
0,0,50.163110,19.412014,23,5,3,2200
1,1,37.336451,58.119225,27,4,7,3600
2,2,46.709232,57.525650,31,6,14,6800
3,3,30.528626,55.379835,26,5,5,3000
4,4,51.521781,35.116755,32,11,6,4600
...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,2200
96,96,30.318396,33.388335,32,5,10,5000
97,97,36.218839,22.235766,32,4,14,6400
98,98,42.936915,38.122442,28,7,5,3400


In [7]:
#Demand values of 2019
demands19=np.array(demand_set['2019'])
demands19

array([15.08853972, 13.45382863, 17.50160494, ...,  3.78593664,
        7.28377172,  7.93874021])

In [8]:
#Initialize the linear optimization model
model1 = LpProblem("DS-ij-Optimization-Model", LpMinimize)
model1

DS-ij-Optimization-Model:
MINIMIZE
None
VARIABLES

In [9]:
#Creating unique index numbers
index_numbers_x = [str(i) for i in range(n_supply*n_demand)]
index_numbers_y = [str(i) for i in range(n_supply)]
index_numbers_x.sort()
index_numbers_y.sort()

In [10]:
DV_variables = LpVariable.matrix("X", index_numbers_x, cat = "Continuous", lowBound= 0 )
dsij_mat19 = np.array(DV_variables).reshape(n_supply,n_demand)
print("Decision Variable/Allocation Matrix: ")
dsij_mat19

Decision Variable/Allocation Matrix: 


array([[X_0, X_1, X_10, ..., X_103680, X_103681, X_103682],
       [X_103683, X_103684, X_103685, ..., X_107367, X_107368, X_107369],
       [X_10737, X_107370, X_107371, ..., X_111052, X_111053, X_111054],
       ...,
       [X_8894, X_88940, X_88941, ..., X_92622, X_92623, X_92624],
       [X_92625, X_92626, X_92627, ..., X_96309, X_9631, X_96310],
       [X_96311, X_96312, X_96313, ..., X_99997, X_99998, X_99999]],
      dtype=object)

In [11]:
Smax19=[]
park_slot=supply_set['total_parking_slots']-(supply_set['existing_num_SCS']+supply_set['existing_num_FCS'])
park_slot.to_numpy()
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,6].values)
    ub=lb+(park_slot[i]*400)
    temp=LpVariable('Y_{}'.format(index_numbers_y[i]),cat='Integer',lowBound=lb,upBound=ub)
    Smax19.append(temp)
Smax19=np.array(Smax19)
Smax19

array([Y_0, Y_1, Y_10, Y_11, Y_12, Y_13, Y_14, Y_15, Y_16, Y_17, Y_18,
       Y_19, Y_2, Y_20, Y_21, Y_22, Y_23, Y_24, Y_25, Y_26, Y_27, Y_28,
       Y_29, Y_3, Y_30, Y_31, Y_32, Y_33, Y_34, Y_35, Y_36, Y_37, Y_38,
       Y_39, Y_4, Y_40, Y_41, Y_42, Y_43, Y_44, Y_45, Y_46, Y_47, Y_48,
       Y_49, Y_5, Y_50, Y_51, Y_52, Y_53, Y_54, Y_55, Y_56, Y_57, Y_58,
       Y_59, Y_6, Y_60, Y_61, Y_62, Y_63, Y_64, Y_65, Y_66, Y_67, Y_68,
       Y_69, Y_7, Y_70, Y_71, Y_72, Y_73, Y_74, Y_75, Y_76, Y_77, Y_78,
       Y_79, Y_8, Y_80, Y_81, Y_82, Y_83, Y_84, Y_85, Y_86, Y_87, Y_88,
       Y_89, Y_9, Y_90, Y_91, Y_92, Y_93, Y_94, Y_95, Y_96, Y_97, Y_98,
       Y_99], dtype=object)

In [12]:
#Defining the Customer Dissatisfaction cost
cost_eq1 = lpSum(dsij_mat19*dist_mat)

In [13]:
#Adding the cost equation to the model
model1 +=  cost_eq1

In [14]:
#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    model1 += lpSum(dsij_mat19[i][j] for j in range(n_demand)) <= Smax19[i] , "Supply Constraints " + str(i)

In [15]:
# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    model1 += lpSum(dsij_mat19[i][j] for i in range(n_supply)) >= demands19[j] , "Demand Constraints " + str(j)

In [16]:
#Solve the model by minimizing the cost equation 
#by following the supply and demand constraints
model1.solve(PULP_CBC_CMD())
status =  LpStatus[model1.status]
print(status)

Optimal


In [17]:
#Total cost of customer dissatisfaction
print("Total Cost:", model1.objective.value())

Total Cost: 1423560.5164511595


In [18]:
dsij_mat19=np.zeros((100,4096))
p=0
lst=[]
for v in model1.variables():
    k=v.value()
    lst.append(k)
    
for i in range(100):
    for j in range(4096):
        dsij_mat19[i][j]=lst[p]
        p+=1
        
dsij_mat19=pd.DataFrame(dsij_mat19)
dsij_mat19

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
dsij_mat19.to_csv('DSij19.csv',index=False)

In [20]:
Smax19=[]
for i in range(100):
    t=lst[p]
    p+=1
    Smax19.append(int(t))
Smax19=np.array(Smax19)
Smax19

array([ 8200, 10000, 11200,  9400, 10600,  9400,  8200, 11600, 10200,
        7200,  4400,  6600,  7800,  9800, 11200,  8600,  4400,  2000,
       10200, 10000,  8000,  9200,  7000,  8800,  6600, 10000, 11000,
       10000, 10600, 10000,  6600, 12000, 12200, 10400,  6600,  3200,
       11200,  4400,  7000, 10400,  8200,  7800,  9800, 12000,  7000,
       11000, 10600,  7400,  9800, 10600, 10600,  4000, 10200, 13200,
       12200,  5800, 10400,  5800,  6400,  7800,  3600,  3800, 11400,
        6000,  7400, 12400,  5200, 11600,  9600, 11000, 11200, 11600,
        7600,  9800,  3800, 12600,  3200, 10200, 12200, 10600,  8400,
       12600,  7600,  9000,  2200,  8600,  9600,  8600,  6000, 11800,
        8200, 13800,  6400,  9200, 10600,  9000, 11800, 12000,  9800,
        9200])

In [21]:
#Checking the Demand constraint (row summation)
print(sum(dsij_mat19[0]))
#Comparing with demand value
print(demands19[0])

15.08854
15.088539719581604


In [22]:
#Checking the Supply constraint (column summation)
print(sum(dsij_mat19.iloc[0,:].values))
#Comparing with supply value
print(Smax19[0])

8199.999978500004
8200


In [23]:
supply_set['Smax19']=Smax19
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Smax18,Smax19
0,0,50.163110,19.412014,23,5,3,2200,8200
1,1,37.336451,58.119225,27,4,7,3600,10000
2,2,46.709232,57.525650,31,6,14,6800,11200
3,3,30.528626,55.379835,26,5,5,3000,9400
4,4,51.521781,35.116755,32,11,6,4600,10600
...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,2200,9000
96,96,30.318396,33.388335,32,5,10,5000,11800
97,97,36.218839,22.235766,32,4,14,6400,12000
98,98,42.936915,38.122442,28,7,5,3400,9800


In [24]:
model2 = LpProblem("Infra-Optimization-Model", LpMinimize)
model2

Infra-Optimization-Model:
MINIMIZE
None
VARIABLES

In [25]:
index_numbers_c = [str(i) for i in range(n_supply)]
index_numbers_c.sort()

In [26]:
S_var=[]
F_var=[]

for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,4].values)
    temp=LpVariable('S_{}'.format(index_numbers_c[i]),cat='Integer', lowBound=lb)
    S_var.append(temp)
S_var=np.array(S_var)

for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,5].values)
    temp=LpVariable('F_{}'.format(index_numbers_c[i]),cat='Integer',lowBound=lb)
    F_var.append(temp)
F_var=np.array(F_var)

tot_p=np.array(supply_set['total_parking_slots'])

In [27]:
#Cost Equation
cost_eq2=lpSum(S_var)+1.5*lpSum(F_var)
model2 += cost_eq2

In [28]:
#Supply Constraint
for i in range(n_supply):
    constraint = (S_var[i]*200)+(F_var[i]*400) == int(supply_set.iloc[i:i+1,-1].values)
    model2 += constraint

In [29]:
model2.solve(PULP_CBC_CMD())
status =  LpStatus[model2.status]
print(status)

Optimal


In [30]:
print("Total Cost:", model2.objective.value()*600)

Total Cost: 2079000.0


In [31]:
# DSij Variable's value
for v in model2.variables():
    print(v.name,"=", v.value())

F_0 = 18.0
F_1 = 23.0
F_10 = 25.0
F_11 = 21.0
F_12 = 21.0
F_13 = 19.0
F_14 = 16.0
F_15 = 26.0
F_16 = 21.0
F_17 = 15.0
F_18 = 8.0
F_19 = 12.0
F_2 = 17.0
F_20 = 21.0
F_21 = 26.0
F_22 = 19.0
F_23 = 9.0
F_24 = 3.0
F_25 = 25.0
F_26 = 22.0
F_27 = 17.0
F_28 = 18.0
F_29 = 15.0
F_3 = 20.0
F_30 = 13.0
F_31 = 22.0
F_32 = 24.0
F_33 = 21.0
F_34 = 21.0
F_35 = 22.0
F_36 = 14.0
F_37 = 27.0
F_38 = 27.0
F_39 = 23.0
F_4 = 14.0
F_40 = 7.0
F_41 = 26.0
F_42 = 9.0
F_43 = 15.0
F_44 = 23.0
F_45 = 17.0
F_46 = 18.0
F_47 = 19.0
F_48 = 27.0
F_49 = 17.0
F_5 = 24.0
F_50 = 25.0
F_51 = 16.0
F_52 = 21.0
F_53 = 23.0
F_54 = 23.0
F_55 = 8.0
F_56 = 24.0
F_57 = 32.0
F_58 = 29.0
F_59 = 13.0
F_6 = 23.0
F_60 = 12.0
F_61 = 12.0
F_62 = 16.0
F_63 = 8.0
F_64 = 8.0
F_65 = 26.0
F_66 = 12.0
F_67 = 17.0
F_68 = 29.0
F_69 = 12.0
F_7 = 25.0
F_70 = 21.0
F_71 = 25.0
F_72 = 27.0
F_73 = 23.0
F_74 = 14.0
F_75 = 22.0
F_76 = 8.0
F_77 = 30.0
F_78 = 6.0
F_79 = 22.0
F_8 = 25.0
F_80 = 23.0
F_81 = 17.0
F_82 = 28.0
F_83 = 16.0
F_84 = 19.0
F_85 = 3.0


In [32]:
p=0
lst=[]
F_var=[]
S_var=[]
for v in model2.variables():
    k=v.value()
    lst.append(int(k))
for i in range(n_supply):
    F_var.append(lst[p])
    p+=1
for i in range(n_supply):
    S_var.append(lst[p])
    p+=1
S_var=np.array(S_var)
F_var=np.array(F_var)
print(S_var)
print(F_var)

[ 5  4  6  5 11  9  9  6  9  6  6  9  5  7  4  5  4  4  1  6  6 10  5  4
  7  6  7  8 11  6  5  6  7  6  5  2  4  4  5  6  7  3 11  6  1  7  3  5
  7  7  7  4  3  2  3  3  6  5  8  7  2  3  5  6  3  4  2  8  6  5  2 12
 10  5  3  3  4  7 11  7  8  7  6  7  5  9  6  5  4  3 11  3  6  8  7  3
  5  4  7  2]
[18 23 25 21 21 19 16 26 21 15  8 12 17 21 26 19  9  3 25 22 17 18 15 20
 13 22 24 21 21 22 14 27 27 23 14  7 26  9 15 23 17 18 19 27 17 24 25 16
 21 23 23  8 24 32 29 13 23 12 12 16  8  8 26 12 17 29 12 25 21 25 27 23
 14 22  8 30  6 22 25 23 17 28 16 19  3 17 21 19 13 28 15 33 13 19 23 21
 27 28 21 22]


In [33]:
supply_set['SCS_19']=S_var
supply_set['FCS_19']=F_var

In [34]:
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Smax18,Smax19,SCS_19,FCS_19
0,0,50.163110,19.412014,23,5,3,2200,8200,5,18
1,1,37.336451,58.119225,27,4,7,3600,10000,4,23
2,2,46.709232,57.525650,31,6,14,6800,11200,6,25
3,3,30.528626,55.379835,26,5,5,3000,9400,5,21
4,4,51.521781,35.116755,32,11,6,4600,10600,11,21
...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,2200,9000,3,21
96,96,30.318396,33.388335,32,5,10,5000,11800,5,27
97,97,36.218839,22.235766,32,4,14,6400,12000,4,28
98,98,42.936915,38.122442,28,7,5,3400,9800,7,21


supply_set.to_csv('Infra_19.csv')